In [24]:
%%capture
!pip install accelerate
!pip install ipywidgets
!pip install --upgrade transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
# prompt: Log in to huggingface
with open("private_/hf_read_token", "r") as f:
  token = f.readline()

hf_read = token

with open("private_/hf_write_token", "r") as f:
  token = f.readline()

hf_write = token

from huggingface_hub import login

login(token=hf_read)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/arjunsohur/.cache/huggingface/token
Login successful


In [26]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Meta-Llama-3-8B-Instruct", device="cuda")

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
def get_hypothesis_generation_system_prompt():
    s = """
    You are a professional debater and philosopher.  You are currently
        studying examples of when people changed their mind and are trying to
        develop certian hypothese to see what arguments are persuasive.

        All of your responses should be one sentence only and should start with
        'Arguments that...' and then continue. Mention nothing of the prompt in
        your response.

        Here are some sample responses:
         - Arguments that are well sourced are more persuasive.
         - Arguments that use date are more persuasive.
         - Arguments that appeal to emotion are more persuasive.

        You are very determined to find the single best hypothesis
    """
    return s

def get_hypothesis_generation_prompt(reply, op):
    s = f"""
        You are a professional debater and philosopher.  You are currently
        studying examples of when people changed their mind and are trying to
        develop certian hypothese to see what arguments are persuasive.  You are
        studying one particular example, at the moment:

        Here's the the orignal poster's arument: {op}

        Here's the reply that changed their mind: {reply}

        Your response derived from this example should be a general hypothesis
        that could be applied to  any argument, meaning you should not discuss
        the example in your response."""

    return s

In [28]:
op = """I can't remember the topic that spurred this discussion, but a friend and I were debating whether man-made things were natural. He took the position that they are unnatural. 

He cited this definition by Merriam-Webster:  existing in nature and not made or caused by people : coming from nature (http://www.merriam-webster.com/dictionary/natural) as his basis for the distinction for natural vs. unnatural.

However, I respectfully disagree with his position and furthermore that definition of natural. People arise from nature. Humankind's capacity to create, problem-solve, analyze, rationalize, and build also come from natural processes. How are the things we create unnatural? It is only through natural occurrences that we have this ability, why is it that we would give the credit of these things solely to man, as opposed to nature? We are not separate from nature, thus, how can any of our actions or creations be unnatural? If we were somehow separate from nature, I would understand the distinction between natural and man-made. However, I think unnatural and man-made are not synonyms by any means. It seems to me that man-made things MUST be natural due to our being part of nature. 

I would love to hear your arguments and to have my view changed if I am mistaken in my logic somewhere along the line.

EDIT: Thanks for your responses lads. As many of you pointed out, our argument depends more on the context of the use of the word and I think that my friend and I were arguing about the semantics and definition more than our intended meaning behind use of the word "natural"."""

In [29]:
resp = """Look at the definition you provided, if we remove the exclusion of things which humans create:

&gt; existing in nature ~~and not made or caused by people~~

So essentially, by this definition, "natural things" are "things that exist," which is frankly rather meaningless. If one wanted to discuss the results of human activity we would then have to make up a new word which could be redefined by the same argument. 

The whole point of the word is to exclude human activity. If you remove that aspect, it simply ceases to have meaning."""

In [30]:
sys = get_hypothesis_generation_system_prompt()
hyp = get_hypothesis_generation_prompt(resp, op)

messages = [
    {"role": "system", "content": sys},
    {"role": "user", "content": hyp},
]

terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [31]:
outputs = pipe(
    messages,
    max_new_tokens=256,
    eos_token_id= pipe.tokenizer.eos_token_id,
    pad_token_id=pipe.tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

outputs

[{'generated_text': [{'role': 'system',
    'content': "\n    You are a professional debater and philosopher.  You are currently\n        studying examples of when people changed their mind and are trying to\n        develop certian hypothese to see what arguments are persuasive.\n\n        All of your responses should be one sentence only and should start with\n        'Arguments that...' and then continue. Mention nothing of the prompt in\n        your response.\n\n        Here are some sample responses:\n         - Arguments that are well sourced are more persuasive.\n         - Arguments that use date are more persuasive.\n         - Arguments that appeal to emotion are more persuasive.\n\n        You are very determined to find the single best hypothesis\n    "},
   {'role': 'user',
    'content': '\n        You are a professional debater and philosopher.  You are currently\n        studying examples of when people changed their mind and are trying to\n        develop certian hypo